In [1]:
import os
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm 
from sklearn.metrics import accuracy_score, classification_report

import torch
import torch.nn.functional as F
from torch.utils import data
from torchinfo import summary
import torch.nn as nn
import torch.optim as optim

# 读取数据

In [2]:
file_dir = "/l1/data/FBDQA2021A_MMP_Challenge_ver0.2/data"

sym = 4
dates = list(range(24))
df = pd.DataFrame()
for date in dates:
    if (date & 1):
        file_name = f"snapshot_sym{sym}_date{date//2}_am.csv"
    else:
        file_name = f"snapshot_sym{sym}_date{date//2}_pm.csv"
    if not os.path.isfile(os.path.join(file_dir,file_name)):
        continue
    new_df = pd.read_csv(os.path.join(file_dir,file_name))
    # 价格+1（从涨跌幅还原到对前收盘价的比例）
    new_df['bid1'] = new_df['n_bid1']+1
    new_df['bid2'] = new_df['n_bid2']+1
    new_df['bid3'] = new_df['n_bid3']+1
    new_df['bid4'] = new_df['n_bid4']+1
    new_df['bid5'] = new_df['n_bid5']+1
    new_df['ask1'] = new_df['n_ask1']+1
    new_df['ask2'] = new_df['n_ask2']+1
    new_df['ask3'] = new_df['n_ask3']+1
    new_df['ask4'] = new_df['n_ask4']+1
    new_df['ask5'] = new_df['n_ask5']+1
    
    # 量价组合
    new_df['spread1'] =  new_df['ask1'] - new_df['bid1']
    new_df['spread2'] =  new_df['ask2'] - new_df['bid2']
    new_df['spread3'] =  new_df['ask3'] - new_df['bid3']
    new_df['mid_price1'] =  new_df['ask1'] + new_df['bid1']
    new_df['mid_price2'] =  new_df['ask2'] + new_df['bid2']
    new_df['mid_price3'] =  new_df['ask3'] + new_df['bid3']
    new_df['weighted_ab1'] = (new_df['ask1'] * new_df['n_bsize1'] + new_df['bid1'] * new_df['n_asize1']) / (new_df['n_bsize1'] + new_df['n_asize1'])
    new_df['weighted_ab2'] = (new_df['ask2'] * new_df['n_bsize2'] + new_df['bid2'] * new_df['n_asize2']) / (new_df['n_bsize2'] + new_df['n_asize2'])
    new_df['weighted_ab3'] = (new_df['ask3'] * new_df['n_bsize3'] + new_df['bid3'] * new_df['n_asize3']) / (new_df['n_bsize3'] + new_df['n_asize3'])
    
    new_df['vol1_rel_diff']   = (new_df['n_bsize1'] - new_df['n_asize1']) / (new_df['n_bsize1'] + new_df['n_asize1'])
    new_df['volall_rel_diff'] = (new_df['n_bsize1'] + new_df['n_bsize2'] + new_df['n_bsize3'] + new_df['n_bsize4'] + new_df['n_bsize5'] \
                     - new_df['n_asize1'] - new_df['n_asize2'] - new_df['n_asize3'] - new_df['n_asize4'] - new_df['n_asize5'] ) / \
                     ( new_df['n_bsize1'] + new_df['n_bsize2'] + new_df['n_bsize3'] + new_df['n_bsize4'] + new_df['n_bsize5'] \
                     + new_df['n_asize1'] + new_df['n_asize2'] + new_df['n_asize3'] + new_df['n_asize4'] + new_df['n_asize5'] )
                            
    new_df['amount'] = new_df['amount_delta'].map(np.log1p)
    df = df.append(new_df)
    




In [3]:
df[['weighted_ab1','weighted_ab2','weighted_ab3', 'vol1_rel_diff', 'volall_rel_diff']].describe()

,weighted_ab1,weighted_ab2,weighted_ab3,vol1_rel_diff,volall_rel_diff
count,47976.000000,47976.000000,47976.000000,47976.000000,47976.000000
mean,1.003776,1.003910,1.004070,0.050167,0.152846
std,0.017222,0.017102,0.016967,0.600878,0.409175
min,0.975636,0.976095,0.976427,-0.999980,-0.913974
25%,0.991279,0.991708,0.992057,-0.460320,-0.153381
50%,0.999439,0.999396,0.999465,0.075037,0.193042
75%,1.012393,1.012740,1.012981,0.582395,0.493286
max,1.072015,1.071591,1.071710,0.999781,0.951323


In [4]:
df

,date,time,sym,n_close,amount_delta,n_midprice,n_bid1,n_bsize1,n_bid2,n_bsize2,...,spread3,mid_price1,mid_price2,mid_price3,weighted_ab1,weighted_ab2,weighted_ab3,vol1_rel_diff,volall_rel_diff,amount
0,0,13:10:03,4,-0.014876,25032.0,-0.015289,-0.015702,2.584610e-05,-0.016529,0.000081,...,0.004132,1.969421,1.969421,1.969421,0.984760,0.985554,0.986383,0.118386,0.626549,10.127950
1,0,13:10:06,4,-0.014876,0.0,-0.015289,-0.015702,2.584610e-05,-0.016529,0.000081,...,0.004132,1.969421,1.969421,1.969421,0.984808,0.985554,0.986383,0.235822,0.650704,0.000000
2,0,13:10:09,4,-0.015702,1669544.0,-0.016116,-0.016529,8.080050e-05,-0.017355,0.000031,...,0.004132,1.967769,1.967769,1.967769,0.984194,0.984272,0.985212,0.749770,0.675229,14.328062
3,0,13:10:12,4,-0.016529,21420.0,-0.016116,-0.016529,8.054896e-05,-0.017355,0.000031,...,0.004132,1.967769,1.967769,1.967769,0.984194,0.984272,0.985212,0.749086,0.669201,9.972127
4,0,13:10:15,4,-0.016529,0.0,-0.016116,-0.016529,8.056831e-05,-0.017355,0.000031,...,0.004132,1.967769,1.967769,1.967769,0.984194,0.984272,0.985217,0.749139,0.669819,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,11,11:19:45,4,-0.004680,1665200.0,-0.003900,-0.004680,1.613688e-05,-0.005460,0.000010,...,0.004680,1.992200,1.992200,1.992200,0.996736,0.995265,0.994582,0.815016,-0.332393,14.325456
1995,11,11:19:48,4,-0.004680,66353.0,-0.003900,-0.004680,1.524684e-05,-0.005460,0.000010,...,0.004680,1.992200,1.992200,1.992200,0.996692,0.995263,0.994582,0.758929,-0.340953,11.102759
1996,11,11:19:51,4,-0.004680,0.0,-0.003900,-0.004680,1.495661e-05,-0.005460,0.000011,...,0.004680,1.992200,1.992200,1.992200,0.996754,0.995279,0.994582,0.838288,-0.338120,0.000000
1997,11,11:19:54,4,-0.003120,28114.0,-0.003510,-0.003900,6.385097e-07,-0.004680,0.000001,...,0.003900,1.992980,1.992980,1.992980,0.996153,0.995398,0.995311,-0.862786,-0.574108,10.244059


In [5]:
feature_col_names = ['bid1','n_bsize1',
                     'bid2','n_bsize2',
                     'bid3','n_bsize3',
                     'bid4','n_bsize4',
                     'bid5','n_bsize5',
                     'ask1','n_asize1',
                     'ask2','n_asize2',
                     'ask3','n_asize3',
                     'ask4','n_asize4',
                     'ask5','n_asize5',
                     'spread1','mid_price1',
                     'spread2','mid_price2',
                     'spread3','mid_price3',
                     'weighted_ab1','weighted_ab2','weighted_ab3','amount',
                     'vol1_rel_diff','volall_rel_diff'
                    ]
label_col_name = ['label_5']

In [6]:
n = len(df)
##划分训练/测试集
train_nums = int(n*0.8)
val_nums = int(n*0.1)
print(f'train_nums: {train_nums}, val_nums: {val_nums}, test_nums: {n-train_nums-val_nums}')
train_data = np.ascontiguousarray(df[feature_col_names][:train_nums].values)
train_label = df[label_col_name][:train_nums].values.reshape(-1)

val_data = np.ascontiguousarray(df[feature_col_names][train_nums:train_nums+val_nums].values)
val_label = df[label_col_name][train_nums:train_nums+val_nums].values.reshape(-1)

test_data = np.ascontiguousarray(df[feature_col_names][train_nums+val_nums:].values)
test_label = df[label_col_name][train_nums+val_nums:].values.reshape(-1)

train_nums: 38380, val_nums: 4797, test_nums: 4799


In [7]:
train_data.shape

(38380, 32)

## GPU准备

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


## 准备dataset

In [9]:
def data_transform(X, T):
    [N, D] = X.shape
    dataX = np.zeros((N - T + 1, T, D))
    for i in range(T, N + 1):
        dataX[i - T] = X[i - T:i, :]
    return dataX

In [10]:
class Dataset(data.Dataset):
    def __init__(self, data, label,  num_classes, T):
        self.T = T

        data = data_transform(data, self.T)

        # self.x = torch.tensor(data).to(torch.float32).unsqueeze(1).to(device)
        self.x = torch.tensor(data).to(torch.float32).to(device)

#         self.y = F.one_hot(torch.tensor(label[T - 1:].astype(np.int64)), num_classes=3)
        self.y = torch.tensor(label[T - 1:].astype(np.int64)).to(device)
    
        self.length = len(self.x)

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        return self.x[index], self.y[index]

In [11]:
batch_size = 512

dataset_train = Dataset(data=train_data,label=train_label, num_classes=3, T=100)
dataset_val   = Dataset(data=val_data,  label=val_label,   num_classes=3, T=100)
dataset_test  = Dataset(data=test_data, label=test_label,  num_classes=3, T=100)

train_loader = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=batch_size, shuffle=True)
val_loader   = torch.utils.data.DataLoader(dataset=dataset_val, batch_size=batch_size, shuffle=False)
test_loader  = torch.utils.data.DataLoader(dataset=dataset_test, batch_size=batch_size, shuffle=False)

print(dataset_train.x.shape, dataset_train.y.shape, dataset_train.x.requires_grad, dataset_train.y.requires_grad,)
print(dataset_val.x.shape, dataset_val.y.shape, dataset_val.x.requires_grad, dataset_val.y.requires_grad,)
print(dataset_test.x.shape, dataset_test.y.shape, dataset_test.x.requires_grad, dataset_test.y.requires_grad,)

torch.Size([38281, 100, 32]) torch.Size([38281]) False False
torch.Size([4698, 100, 32]) torch.Size([4698]) False False
torch.Size([4700, 100, 32]) torch.Size([4700]) False False


In [12]:
print(dataset_train.x.device)
print(dataset_train.y.device)

cpu
cpu


# 定义模型

In [13]:
class deeplob(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.num_classes = num_classes
        
        # convolution blocks
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(1,2), stride=(1,2)),
            nn.LeakyReLU(negative_slope=0.01),
#             nn.Tanh(),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(5,1),stride=(2,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(1,2), stride=(1,2)),
            nn.Tanh(),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.Tanh(),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1),stride=(2,1)),
            nn.Tanh(),
            nn.BatchNorm2d(32),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(1,8)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(4,1),stride=(2,1)),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(32),
        )
        
        # inception moduels
        self.inp1 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(1,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels=64, out_channels=16, kernel_size=(3,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(16),
        )
        self.inp2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(1,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(64),
            nn.Conv2d(in_channels=64, out_channels=16, kernel_size=(5,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(16),
        )
        self.inp3 = nn.Sequential(
            nn.MaxPool2d((3, 1), stride=(1, 1), padding=(1, 0)),
            nn.Conv2d(in_channels=32, out_channels=16, kernel_size=(1,1), padding='same'),
            nn.LeakyReLU(negative_slope=0.01),
            nn.BatchNorm2d(16),
        )
       
        # lstm layers
        self.fc = nn.Sequential(nn.Linear(384, 64),nn.Linear(64, self.num_classes))

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)

        x_inp1 = self.inp1(x)
        x_inp2 = self.inp2(x)
        x_inp3 = self.inp3(x)

        x = torch.cat((x_inp1, x_inp2, x_inp3), dim=1)

        x = x.reshape(-1,48*8)
        x = self.fc(x)

        forecast_y = torch.softmax(x, dim=1)

        return forecast_y

In [37]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.1)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h0, c0))
        
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        
        return F.softmax(out, dim=1)

# 模型参数定义
input_size = 32
hidden_size = 64
num_layers = 1
num_classes = 3

In [38]:
model_cnn = deeplob(num_classes = 3)
model_cnn.to(device);
model_lstm = LSTMModel(input_size, hidden_size, num_layers, num_classes).to(device)
model_lstm.to(device);

In [39]:
# summary(model, (234, 1, 100, 32))
summary(model_lstm, (1, 100, 32))

Layer (type:depth-idx)                   Output Shape              Param #
LSTMModel                                [1, 3]                    --
├─LSTM: 1-1                              [1, 100, 64]              25,088
├─Dropout: 1-2                           [1, 64]                   --
├─Linear: 1-3                            [1, 3]                    195
Total params: 25,283
Trainable params: 25,283
Non-trainable params: 0
Total mult-adds (M): 2.51
Input size (MB): 0.01
Forward/backward pass size (MB): 0.05
Params size (MB): 0.10
Estimated Total Size (MB): 0.17

# 训练模型

In [17]:
criterion_cnn = nn.CrossEntropyLoss()
optimizer_cnn = torch.optim.SGD(model_cnn.parameters(), lr=0.001, momentum=0.9, weight_decay = 1e-5)

def batch_gd_cnn(model, criterion, optimizer, train_loader, test_loader, epochs):
    train_losses = np.zeros(epochs)
    test_losses = np.zeros(epochs)
    best_test_loss = np.inf
    best_test_epoch = 0

    for it in tqdm(range(epochs)):
        if ((epochs+1) % 10 == 0):
            optimizer.lr = optimizer.lr*0.5
        model.train()
        t0 = datetime.now()
        train_loss = []
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            
            outputs = model(inputs)

            loss = criterion(outputs, targets)

            loss.backward()
            
            optimizer.step()
            
            train_loss.append(loss.item())
            
        # Get train loss and test loss
        train_loss = np.mean(train_loss) # a little misleading
    
        model.eval()
        test_loss = []
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)      
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss.append(loss.item())
        test_loss = np.mean(test_loss)

        # Save losses
        train_losses[it] = train_loss
        test_losses[it] = test_loss
        
        if test_loss < best_test_loss:
            torch.save(model, f'best_val_model_pytorch_sym{sym}_date{dates[-1]}')
            best_test_loss = test_loss
            best_test_epoch = it
            print('model saved')

        dt = datetime.now() - t0
        print(f'Epoch {it+1}/{epochs}, Train Loss: {train_loss:.4f}, \
          Validation Loss: {test_loss:.4f}, Duration: {dt}, Best Val Epoch: {best_test_epoch}')
    torch.save(model, f'final_model_pytorch_sym{sym}_date{dates[-1]}')
    return train_losses, test_losses

In [18]:
criterion_lstm = nn.CrossEntropyLoss()
optimizer_lstm = optim.Adam(model_lstm.parameters(), lr=0.01)

def batch_gd_lstm(model, criterion, optimizer, train_loader, test_loader, epochs):
    train_losses = np.zeros(epochs)
    test_losses = np.zeros(epochs)
    best_test_loss = np.inf
    best_test_epoch = 0

    for it in tqdm(range(epochs)):
        model.train()
        t0 = datetime.now()
        train_loss = []
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            inputs, targets = inputs.to(device), targets.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
        
        train_loss = np.mean(train_loss)
    
        model.eval()
        test_loss = []
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                test_loss.append(loss.item())
        test_loss = np.mean(test_loss)

        train_losses[it] = train_loss
        test_losses[it] = test_loss
        
        if test_loss < best_test_loss:
            torch.save(model.state_dict(), f'best_val_model_lstm_sym{sym}_date{dates[-1]}.pth')
            best_test_loss = test_loss
            best_test_epoch = it
            print('model saved')

        dt = datetime.now() - t0
        print(f'Epoch {it+1}/{epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {test_loss:.4f}, Duration: {dt}, Best Val Epoch: {best_test_epoch}')
    
    torch.save(model.state_dict(), f'final_model_lstm_sym{sym}_date{dates[-1]}.pth')
    return train_losses, test_losses


In [19]:
for inputs, targets in train_loader:
    print('inputs.shape:', inputs.shape)
    print('targets.shape:', targets.shape)
    outputs = model_lstm(inputs)
    print('outputs.shape:', outputs.shape)
    break

inputs.shape: torch.Size([512, 100, 32])
targets.shape: torch.Size([512])
outputs.shape: torch.Size([512, 3])


In [20]:
train_losses, val_losses = batch_gd_cnn(model_cnn, criterion_cnn, optimizer_cnn, 
                                    train_loader, val_loader, epochs=50)

  0%|          | 0/50 [00:00<?, ?it/s]


RuntimeError: Given groups=1, weight of size [32, 1, 1, 2], expected input[1, 512, 100, 32] to have 1 channels, but got 512 channels instead

In [21]:
train_losses, val_losses = batch_gd_lstm(model_lstm, criterion_lstm, optimizer_lstm, train_loader, val_loader, epochs=50)

  2%|▏         | 1/50 [00:18<15:14, 18.67s/it]

model saved
Epoch 1/50, Train Loss: 0.9327, Validation Loss: 0.9137, Duration: 0:00:18.672007, Best Val Epoch: 0


  4%|▍         | 2/50 [00:37<14:53, 18.60s/it]

model saved
Epoch 2/50, Train Loss: 0.9289, Validation Loss: 0.9137, Duration: 0:00:18.443648, Best Val Epoch: 1


  6%|▌         | 3/50 [00:55<14:25, 18.42s/it]

...

 20%|██        | 10/50 [03:01<11:58, 17.96s/it]

model saved
Epoch 10/50, Train Loss: 0.9290, Validation Loss: 0.9137, Duration: 0:00:18.088701, Best Val Epoch: 9


 22%|██▏       | 11/50 [03:19<11:39, 17.94s/it]

model saved
Epoch 11/50, Train Loss: 0.9290, Validation Loss: 0.9137, Duration: 0:00:17.909127, Best Val Epoch: 10


KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(16,6))
plt.plot(train_losses, label='train loss')
plt.plot(val_losses, label='validation loss')
plt.legend()

# 测试模型

In [ ]:
model_cnn = torch.load(f'best_val_model_pytorch_sym{sym}_date{dates[-1]}', map_location=device)
model_cnn.eval()
all_targets = []
all_predictions = []

for inputs, targets in test_loader:
    # Move to GPU
    inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)

    # Forward pass
    outputs = model_cnn(inputs)
    
    # Get prediction
    # torch.max returns both max and argmax
    _, predictions = torch.max(outputs, 1)

    all_targets.append(targets.cpu().numpy())
    all_predictions.append(predictions.cpu().numpy())

all_targets = np.concatenate(all_targets)    
all_predictions = np.concatenate(all_predictions)    
print('accuracy_score:', accuracy_score(all_targets, all_predictions))
print(classification_report(all_targets, all_predictions, digits=4))

In [ ]:
# model = torch.load('best_val_model_pytorch',map_location=device)
all_targets = []
all_predictions = []

for inputs, targets in train_loader:

    # Forward pass
    outputs = model_cnn(inputs)
    
    # Get prediction
    # torch.max returns both max and argmax
    _, predictions = torch.max(outputs, 1)

    all_targets.append(targets.cpu().numpy())
    all_predictions.append(predictions.cpu().numpy())

all_targets = np.concatenate(all_targets)    
all_predictions = np.concatenate(all_predictions) 
print('accuracy_score:', accuracy_score(all_targets, all_predictions))
print(classification_report(all_targets, all_predictions, digits=4))

# 过拟合：小规模数据难以充分训练大模型

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP,self).__init__()
        self.net = nn.Sequential(
                    nn.Linear(3200,128),
                    nn.LeakyReLU(),
                    nn.Linear(128,64),
                    nn.LeakyReLU(),
                    nn.Linear(64,64),
                    nn.LeakyReLU(),
                    nn.Linear(64,3)
                )
        
    def forward(self,x):
        x = x.view(-1,100*32)
        x = self.net(x)
        return torch.softmax(x, dim=1)

In [ ]:
model_cnn = MLP()
model_cnn.to(device)
summary(model_cnn,(1,1,100,32))

In [ ]:
criterion = nn.CrossEntropyLoss(weight = torch.Tensor([2,1,2]).to(device))
optimizer = torch.optim.AdamW(model_cnn.parameters(), lr=0.001)#, momentum=0.9, weight_decay = 1e-5)

In [ ]:
train_losses, val_losses = batch_gd_cnn(model_cnn, criterion, optimizer, 
                                    train_loader, val_loader, epochs=500)

In [ ]:
model_cnn = torch.load(f'best_val_model_pytorch_sym{sym}_date{dates[-1]}', map_location=device)

all_targets = []
all_predictions = []

for inputs, targets in test_loader:
    # Move to GPU
    inputs, targets = inputs.to(device, dtype=torch.float), targets.to(device, dtype=torch.int64)

    # Forward pass
    outputs = model_cnn(inputs)
    
    # Get prediction
    # torch.max returns both max and argmax
    _, predictions = torch.max(outputs, 1)

    all_targets.append(targets.cpu().numpy())
    all_predictions.append(predictions.cpu().numpy())

all_targets = np.concatenate(all_targets)    
all_predictions = np.concatenate(all_predictions)    
print('accuracy_score:', accuracy_score(all_targets, all_predictions))
print(classification_report(all_targets, all_predictions, digits=4))